# Reproduce PEFT(Parameter-Efficient Fine-Tuning) on Pre-trained Model(Geneformer) with Identification
In this tutorial, we demonstrate how to reproduce a PEFT (Parameter-Efficient Fine-Tuning) pre-trained model(Geneformer) on a specific dataset for the identification cell type task. This tutorial serves as a practical example. There are two steps that need to be executed.

## Step 1: Modify the parameters
### There are four key settings that the reader needs to modify. The available options are listed below:
***dataset_name*** : NSCLC/COVID/ms, \
***data_path***: {data_path}, \
***output_path***: {output_path}, \
***model_path***: {checkpoint_path},



In [ ]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--dataset_name", type=str, default='ms',help='NSCLC/COVID/ms')
parser.add_argument("--data_path", type=str, default='../data/celltype_identification/', help='Path of data for predicting')
parser.add_argument("--output_path", type=str, default=f"../Geneformer/", help='output data path.')
parser.add_argument("--model_path", type=str, default=f"../checkpoint/celltype_identification/COVID/Encoder_adapter/", help='Path of data for predicting.')

## Step 2: Start the validation

In [8]:

# %%

import os
import sys
from pathlib import Path

sys.path.append("..")
import datasets
import scanpy as sc
import pandas as pd
from accelerate.utils import is_datasets_available
from torch.utils.data import DataLoader

from transformerslocal.src.transformers.trainer_utils import seed_worker

from numpy import mean

import torch

from transformerslocal.src.transformers import EarlyStoppingCallback

# device
# device = torch.device("cuda:3")
# import
import torch.nn as nn
# imports
from collections import Counter
import datetime
import pickle
import subprocess
import seaborn as sns;

sns.set()
from datasets import load_from_disk
from sklearn.metrics import accuracy_score, f1_score, balanced_accuracy_score, precision_score, recall_score
from transformerslocal.src.transformers.models.bert.modeling_bert import BertForSequenceClassification
# from transformers  import BertForSequenceClassification
from transformers import Trainer
from transformers.training_args import TrainingArguments
from geneformer import DataCollatorForCellClassification
import numpy as np
import loralib as lora



parser.add_argument("--epoch", type=int, default=100, help='Number of epochs.')
parser.add_argument("--use_prompt", type=bool, default=True, help='whether use prompt or not.')
parser.add_argument("--lr", type=int, default=1e-4, help='')
parser.add_argument("--batch_size", type=int, default=10, help='')
args, unknown = parser.parse_known_args()

In [9]:

dataset_name=args.dataset_name
model_path = args.model_path
data_dir =args.data_path
accuracys=[]
precisions=[]
recalls=[]
macro_f1s=[]
mic_precisions=[]
mic_recalls=[]
micro_f1s=[]
if dataset_name=="MergedHuman":
    n_splits = 4
else:
    n_splits = 5

i=1

# for i in range(n_splits):

train_data_dir = f'{data_dir}/{dataset_name}/{i}/{dataset_name}_train{i}.dataset'
eval_data_dir = f'{data_dir}/{dataset_name}/{i}/{dataset_name}_val{i}.dataset'
test_data_dir = f'{data_dir}/{dataset_name}/{i}/{dataset_name}_test{i}.dataset'
train_dataset = load_from_disk(train_data_dir)
eval_dataset = load_from_disk(eval_data_dir)
test_dataset = load_from_disk(test_data_dir)

dataset_list = []
evalset_list = []
organ_list = []
testset_list = []
target_dict_list = []

trainset_organ = train_dataset
evalset_organ = eval_dataset
testset_organ = test_dataset
# per scDeepsort published method, drop cell types representing <0.5% of cells
celltype_counter = Counter(trainset_organ["cell_type"])
total_cells = sum(celltype_counter.values())
cells_to_keep = [k for k, v in celltype_counter.items() if v > (0.005 * total_cells)]

trainset_organ_subset = trainset_organ
evalset_organ_subset = evalset_organ
testset_organ_subset = testset_organ

# shuffle datasets and rename columns
trainset_organ_shuffled = trainset_organ_subset.shuffle(seed=42)
trainset_organ_shuffled = trainset_organ_shuffled.rename_column("cell_type", "label")

evalset_organ_shuffled = evalset_organ_subset.shuffle(seed=42)
evalset_organ_shuffled = evalset_organ_shuffled.rename_column("cell_type", "label")

testset_organ_shuffled = testset_organ_subset.shuffle(seed=42)
testset_organ_shuffled = testset_organ_shuffled.rename_column("cell_type", "label")

# create dictionary of cell types : label ids
target_names = list(Counter(trainset_organ_shuffled["label"] + evalset_organ_shuffled["label"] + testset_organ_shuffled["label"]).keys())
target_name_id_dict = dict(zip(target_names, [i for i in range(len(target_names))]))
target_dict_list += [target_name_id_dict]
target_name_id_celltype_dict = {value: key for key, value in target_name_id_dict.items()}

# change labels to numerical ids
def classes_to_ids(example):
    example["label"] = target_name_id_dict[example["label"]]
    return example

labeled_trainset = trainset_organ_shuffled.map(classes_to_ids, num_proc=16)
labeled_evalset = evalset_organ_shuffled.map(classes_to_ids, num_proc=16)
labeled_testset = testset_organ_shuffled.map(classes_to_ids, num_proc=16)

labeled_train_split = labeled_trainset
labeled_eval_split = labeled_evalset

labeled_test_split_subset = labeled_testset
trained_labels = list(Counter(labeled_train_split["label"]).keys())
labeled_eval_split_subset = labeled_eval_split

dataset_list += [labeled_train_split]
evalset_list += [labeled_eval_split_subset]
testset_list += [labeled_test_split_subset]
# %%

trainset_dict = dataset_list
traintargetdict_dict = target_dict_list
evalset_dict = evalset_list
testset_dict = testset_list

In [10]:

def SaveProcessResult(labels, pred_cellcodes, confidence_socre):
    d_save = {'pred_celltype': [target_name_id_celltype_dict[pred_cellcode] for pred_cellcode in pred_cellcodes],
              'gt_celltype': [target_name_id_celltype_dict[label] for label in labels],
              'pred_cellcode': pred_cellcodes,
              'gt_cellcode': labels,
              'confidence_socre': confidence_socre
              }
    pd.DataFrame(data=d_save).to_csv(f'{output_dir}/Geneformer_dataset{dataset_name}_result.csv')


def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # save confidence score and label to csv
    confidence_score = nn.Softmax(dim=-1)(torch.from_numpy(pred.predictions)).detach().cpu().numpy().tolist()
    labels_tocsv = pred.label_ids.tolist()
    SaveProcessResult(labels_tocsv, preds, confidence_score)

    # calculate accuracy and macro f1 using sklearn's function
    acc = balanced_accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='macro')
    recall = recall_score(labels, preds, average='macro')
    macro_f1 = f1_score(labels, preds, average='macro')

    micro_precision = precision_score(labels, preds, average='micro')
    micro_recall = recall_score(labels, preds, average='micro')
    micro_f1 = f1_score(labels, preds, average='micro')
    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'macro_f1': macro_f1,
        "micro_precision": micro_precision,
        "micro_recall": micro_recall,
        "micro_f1": micro_f1

    }


# set model parameters
# max input size
max_input_size = 2 ** 11  # 2048
max_lr = args.lr
# how many pretrained layers to freeze
freeze_layers = 0
# number gpus
num_gpus = 1
# number cpu cores
num_proc = 16
# batch size for training and eval
geneformer_batch_size =args.batch_size
# learning schedule
lr_schedule_fn = "linear"
# warmup steps
warmup_steps = 500
# number of epochs
epochs = args.epoch
# optimizer
optimizer = "adamw"


# %%
celltype_id_labels = [target_name_id_dict[item] for item in train_dataset["cell_type"]]
class_num = np.unique(celltype_id_labels, return_counts=True)[1].tolist()
class_weight = torch.tensor([(1 - (x / sum(class_num))) ** 2 for x in class_num])
weighted_loss = nn.CrossEntropyLoss(weight=class_weight)

organ_trainset = trainset_dict[0]
organ_evalset = evalset_dict[0]
organ_testset = testset_dict[0]
organ_label_dict = traintargetdict_dict[0]
logging_steps = 1

In [11]:
model = BertForSequenceClassification.from_pretrained(
        # "../Pretrain_ckpts/geneformer-12L-30M-prompt",
        model_path+str(i),
        num_labels=len(organ_label_dict.keys()),
        output_attentions=False,
        output_hidden_states=False,
        ignore_mismatched_sizes=True).to("cuda")
prompt_type = model.config.prompt_type
output_dir = args.output_path + prompt_type + '/' + dataset_name + f"/{i}"
model.config.save_pretrained(output_dir)
print(output_dir)
pre_freeze_param_count = sum(dict((p.data_ptr(), p.numel()) for p in model.parameters()).values())

keywords = [
    'lora_A',
    'lora_B',
    'lora_value.bias',
    'lora_key.bias',
    'lora_query.bias',
    'prompt_embeddings',
    'adapter',
    'Adapter'
]

for name, para in model.named_parameters():
    para.requires_grad = False
for param in model.classifier.parameters():
    param.requires_grad = True
for param in model.bert.pooler.parameters():
    param.requires_grad = True
for name, para in model.named_parameters():
    if any(keyword in name for keyword in keywords):
        para.requires_grad = True

print("-" * 89)
learnable_params = {k: v for k, v in model.named_parameters() if v.requires_grad == True}
for k, v in learnable_params.items():
    print(f"Learnable params {k} with shape {v.shape}")

post_freeze_param_count = sum(
    dict((p.data_ptr(), p.numel()) for p in model.parameters() if p.requires_grad).values())
print("Total Pre freeze Params: %.2fM" % (pre_freeze_param_count / 1e6,))
print("Total Post freeze Params: %.2fM" % (post_freeze_param_count / 1e6,))

/users/PCON0022/yuy702/Geneformer/encoder_prompt/ms/1
-----------------------------------------------------------------------------------------
Learnable params bert.encoder.layer.0.attention.Space_Adapter.D_fc1.weight with shape torch.Size([128, 512])
Learnable params bert.encoder.layer.0.attention.Space_Adapter.D_fc1.bias with shape torch.Size([128])
Learnable params bert.encoder.layer.0.attention.Space_Adapter.D_fc2.weight with shape torch.Size([512, 128])
Learnable params bert.encoder.layer.0.attention.Space_Adapter.D_fc2.bias with shape torch.Size([512])
Learnable params bert.encoder.layer.1.attention.Space_Adapter.D_fc1.weight with shape torch.Size([128, 512])
Learnable params bert.encoder.layer.1.attention.Space_Adapter.D_fc1.bias with shape torch.Size([128])
Learnable params bert.encoder.layer.1.attention.Space_Adapter.D_fc2.weight with shape torch.Size([512, 128])
Learnable params bert.encoder.layer.1.attention.Space_Adapter.D_fc2.bias with shape torch.Size([512])
Learnable pa

In [13]:

class MyTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        if model.config.prompt_type == "prefix_prompt":
            mask_cat = torch.ones([inputs["input_ids"].shape[0], model.config.num_token]).to("cuda")
            mask_middle = torch.cat((inputs["attention_mask"][:, :1], mask_cat), dim=1)
            inputs["attention_mask"] = torch.cat((mask_middle, inputs["attention_mask"][:, 1:]), dim=1)

        outputs = model(**inputs)
        loss_fct = weighted_loss.to(outputs['logits'].device)
        loss = loss_fct(outputs['logits'], inputs['labels'])

        return (loss, outputs) if return_outputs else loss

    def get_train_dataloader(self) -> DataLoader:
        """
        Returns the training [`~torch.utils.data.DataLoader`].

        Will use no sampler if `train_dataset` does not implement `__len__`, a random sampler (adapted to distributed
        training if necessary) otherwise.

        Subclass and override this method if you want to inject some custom behavior.
        """
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")

        train_dataset = self.train_dataset
        data_collator = self.data_collator
        if is_datasets_available() and isinstance(train_dataset, datasets.Dataset):
            train_dataset = self._remove_unused_columns(train_dataset, description="training")
        else:
            data_collator = self._get_collator_with_removed_columns(data_collator, description="training")

        dataloader_params = {
            "batch_size": self._train_batch_size,
            "collate_fn": data_collator,
            "num_workers": self.args.dataloader_num_workers,
            "pin_memory": self.args.dataloader_pin_memory,
        }

        if not isinstance(train_dataset, torch.utils.data.IterableDataset):
            train_class_num = np.unique(train_dataset["label"], return_counts=True)[1]
            sample_weights = 1.0 / train_class_num[train_dataset["label"]]
            sample_weights = sample_weights / np.sum(sample_weights)
            train_num = train_dataset.shape[0]
            train_sampler = torch.utils.data.sampler.WeightedRandomSampler(sample_weights, train_num,
                                                                           replacement=True)
            dataloader_params["sampler"] = train_sampler
            dataloader_params["drop_last"] = self.args.dataloader_drop_last
            dataloader_params["worker_init_fn"] = seed_worker

        return self.accelerator.prepare(DataLoader(train_dataset, **dataloader_params))

current_date = datetime.datetime.now()
datestamp = f"{str(current_date.year)[-2:]}-{current_date.month:02d}-{current_date.day:02d}_{current_date.hour:02d}:{current_date.minute:02d}:{current_date.second:02d}"

# ensure not overwriting previously saved model
saved_model_test = os.path.join(output_dir, f"pytorch_model.bin")
model.config.save_pretrained(output_dir)
# print(datestamp)
if os.path.isfile(saved_model_test) == True:
    raise Exception("Model already saved to this directory.")

# make output directory
subprocess.call(f'mkdir {output_dir}', shell=True)
# set training arguments
training_args = {
    "learning_rate": max_lr,
    "do_train": True,
    "do_eval": True,
    "evaluation_strategy": "epoch",
    "save_strategy": "epoch",
    "logging_steps": logging_steps,
    "group_by_length": True,
    "length_column_name": "length",
    "disable_tqdm": False,
    "lr_scheduler_type": lr_schedule_fn,
    "warmup_steps": warmup_steps,
    "weight_decay": 0.001,
    "per_device_train_batch_size": geneformer_batch_size,
    "per_device_eval_batch_size": geneformer_batch_size,
    "num_train_epochs": epochs,
    "load_best_model_at_end": True,
    "output_dir": output_dir,
    "save_safetensors": False,
}

training_args_init = TrainingArguments(**training_args)

# create the trainer
trainer = MyTrainer(
    model=model,
    args=training_args_init,
    data_collator=DataCollatorForCellClassification(),
    train_dataset=organ_trainset,
    eval_dataset=organ_evalset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)
# train the cell type classifier
# trainer.train()
predictions = trainer.predict(organ_testset)

print(f"fold{i}:"
      f"macro Accuracy: {predictions.metrics['test_accuracy']:.3f}, macro Precision: {predictions.metrics['test_precision']:.3f},macro Recall: {predictions.metrics['test_recall']:.3f}, "
      f"macro F1: {predictions.metrics['test_macro_f1']:.3f}"
      )
print(f"fold{i}:"
      f"micro Accuracy: {predictions.metrics['test_accuracy']:.3f}, micro Precision: {predictions.metrics['test_micro_precision']:.3f}, micro Recall: {predictions.metrics['test_micro_recall']:.3f}, "
      f"micro F1: {predictions.metrics['test_micro_f1']:.3f}")

with open(f"{output_dir}/predictions.pickle", "wb") as fp:
    pickle.dump(predictions, fp)
trainer.save_metrics("eval", predictions.metrics)
trainer.save_model(output_dir)


mkdir: cannot create directory ‘/users/PCON0022/yuy702/Geneformer/encoder_prompt/ms/1’: File exists
/fs/ess/PAS1475/Yang/yy/env/Geneformer/lib/python3.8/site-packages/accelerate/accelerator.py:451: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(
Detected kernel version 3.10.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
/users/PCON0022/yuy702/Geneformer/example_py/../geneformer/collator_for_classification.py:582: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k:

/users/PCON0022/yuy702/Geneformer/example_py/../geneformer/collator_for_classification.py:582: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batch = {k: torch.tensor(v, dtype=torch.int64) for k, v in batch.items()}


fold1:macro Accuracy: 0.842, macro Precision: 0.790,macro Recall: 0.842, macro F1: 0.800
fold1:micro Accuracy: 0.842, micro Precision: 0.829, micro Recall: 0.829, micro F1: 0.829
